In [2]:
# # azureml-core of version 1.0.72 or higher is required
# # azureml-core of version 1.0.72 or higher is required
# from azureml.core import Workspace, Dataset

# subscription_id = '3f2ab3f5-468d-4ba7-bc14-9d3a9da4bcc5'
# resource_group = 'TownsendAML4'
# workspace_name = 'townsendws4'

# workspace = Workspace(subscription_id, resource_group, workspace_name)

# dataset = Dataset.get_by_name(workspace, name='Orca3_v2_epoch_3')
# dataset.download(target_path='/data/arindam/Orca3_v2_epoch_3', overwrite=False)

In [8]:
import torch
import transformers
import datasets
from tqdm import tqdm
import torch
import os 


import copy
from dataclasses import dataclass, field
import json
import logging
import pathlib
from typing import Dict, Optional, Sequence
import time
import torch
import os

from torch.utils.data import Dataset
from transformers import Trainer, HfArgumentParser, TrainingArguments
from transformers import StoppingCriteriaList, StoppingCriteria


class EOSStoppingCriteria(StoppingCriteria):
    def __init__(self, stops=[], encounters=1):
        super().__init__()
        self.stops = stops

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        last_token = input_ids[0][-1]
        for stop in self.stops:
            if stop == last_token:
                return True
        return False

def to_prompt(messages):
    text = ''
    for message in messages:
        text += f"<|im_start|>{message['role']}\n{message['content']}\n<|im_end|>\n"
    text += f"<|im_start|>assistant\n"
    return text

local_rank = 0
model = transformers.AutoModelForCausalLM.from_pretrained(
    "../../orca3/Orca3_v2_epoch_3/",
    torch_dtype=torch.bfloat16
)
model = model.to(local_rank)
model.eval()

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.09it/s]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32003, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [9]:
sm = """
You are AutoGPT, you can use many tools(functions) to do the following task.
First I will give you the task description, and your task start.
At each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:
Thought: you should always think about what to do
Action: the action to take (name of the api)
Action Input: the input to the action

After the call, you will get the call result, and you are now in a new state.
Then you will analyze your status now, then decide what to do next...
After many (Thought-call) pairs, you finally perform the task, then you can give your finial answer. If you believe that you have obtained enough information (which can be judge from the history observations) that can answer the task, please call:
Action: Finish
Action Input: {{"return_type": "give_answer", "final_answer": your answer string}}.
Remember: 
1.the state change is irreversible, you can't go back to one of the former state, if you want more information, output your request for additional information.
2.All the thought is short, at most in 3-5 sentences.
3.You can do more then one trys, so if your plan is to continusly try some conditions, you can do one of the conditions per try.
4.ALWAYS call "Finish" function at the end of the task. And the final answer should contain enough information to show to the user,If you can't handle the task, or you find that function calls always fail(the function is not valid now), use function Finish->give_up_and_restart.

Let's Begin!
Specifically, you have access to the following APIs: 
[{'name': 'love_quote_for_olato_quotes', 'description': 'This is the subfunction for tool "olato_quotes", you can use this tool.The description of this function is: "It shows random quotes"', 'parameters': {'type': 'object', 'properties': {}, 'required': [], 'optional': []}}, {'name': 'success_quote_for_olato_quotes', 'description': 'This is the subfunction for tool "olato_quotes", you can use this tool.The description of this function is: "It shows random quotes"', 'parameters': {'type': 'object', 'properties': {}, 'required': [], 'optional': []}}, {'name': 'motivation_quote_for_olato_quotes', 'description': 'This is the subfunction for tool "olato_quotes", you can use this tool.The description of this function is: "It shows random quotes"', 'parameters': {'type': 'object', 'properties': {}, 'required': [], 'optional': []}}, {'name': 'Finish', 'description': 'If you believe that you have obtained a result that can answer the task, please call this function to provide the final answer. Alternatively, if you recognize that you are unable to proceed with the task in the current state, call this function to restart. Remember: you must ALWAYS call this function at the end of your attempt, and the only part that will be shown to the user is the final answer, so it should contain sufficient information.', 'parameters': {'type': 'object', 'properties': {'return_type': {'type': 'string', 'enum': ['give_answer', 'give_up_and_restart']}, 'final_answer': {'type': 'string', 'description': 'The final answer you want to give the user. You should have this field if "return_type"=="give_answer"'}}, 'required': ['return_type']}}]
"""

prompt = """
I'm planning a surprise party for my best friend, and I want to include meaningful quotes in the decorations. Can you provide me with random love, success, and motivation quotes? It would be great to have quotes that can celebrate love, success, and inspire everyone at the party. Thank you so much for your help!
"""

In [ ]:

tokenizer = transformers.AutoTokenizer.from_pretrained(
    "../../orca3/Orca3_v2_epoch_3/",
    model_max_length=5000,
    padding_side="right"
)
tokenizer.pad_token = tokenizer.eos_token

res =  []
gen_args = {"num_return_sequences":1}
# if data_args.do_sample:
#     gen_args["top_p"] = 0.95
#     gen_args["temperature"] = 0.7


stop_word_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(["<|im_end|>"])[0]]
stopping_criteria = StoppingCriteriaList([EOSStoppingCriteria(stops=stop_word_ids)])

messages = [{"role":"system", "content":sm}, {"role":"user", "content":prompt}]
text = to_prompt(messages)
input_ids = tokenizer(tokenizer.bos_token + text, add_special_tokens=False, padding=False,
                               return_tensors='pt').input_ids
input_ids = input_ids.to(local_rank)
output_ids = model.generate(input_ids, max_length=5000, do_sample=False, use_cache=True,
                                        stopping_criteria=stopping_criteria,  **gen_args)
output = [tokenizer.decode(oid[len(input_ids[0]):], skip_special_tokens=True).strip() for oid in output_ids]

In [5]:
print(output[0])

Thought: The user has requested random quotes for different themes: love, success, and motivation. I have access to three subfunctions of the "olato_quotes" tool that can provide these quotes. I will start by retrieving a random love quote.

Action: love_quote_for_olato_quotes
Action Input: {}
